## Create Deployment

In this notebook, we'll create a deployment for our recommendation system.

**NOTE Currently the transformer scripts are not implemented.**

### User Model Deployment

We start by deploying our user/query model.

In [ ]:
import hsml

conn = hsml.connection()
mr = conn.get_model_registry()

user_model = mr.get_model("query_model")
user_model_deployment = user_model.deploy(name="querymodel", serving_tool="KSERVE")

At this point, we have registered our deployment. To start it up we need to run:

In [ ]:
user_model_deployment.start()

We can test the deployment by making a prediction on the input example we registered together with the model.

In [ ]:
user_model_deployment.predict({"instances" : user_model.input_example})

Let's stop the deployment when we're not using it.

In [ ]:
user_model_deployment.stop()

### Ranking Model Deployment

Next, we'll deploy our ranking model. Since it is a CatBoost model we need to implement a `Predict` class that tells Hopsworks how to load the model and how to use it.

In [ ]:
%%writefile ranking_predictor.py

import joblib
import os

class Predict(object):
    
    def __init__(self):
        """ Initializes the serving state, reads a model from HDFS"""
        # NOTE: env var ARTIFACT_FILES_PATH has the path to the model artifact files
        
        self.model = joblib.load(os.environ["ARTIFACT_FILES_PATH"] + "/ranking_model.pkl")


    def predict(self, inputs):
        """ Serves a prediction request usign a trained model"""
        return self.model.predict_proba(inputs).tolist()

This script must be uploaded to the cluster.

In [ ]:
import os
import hopsworks

hopsworks_conn = hopsworks.connection()
project = hopsworks_conn.get_project()
dataset_api = project.get_dataset_api()

uploaded_file_path = dataset_api.upload("ranking_predictor.py", "Models", overwrite=True)
predictor_script_path = os.path.join("/Projects", project.name, uploaded_file_path)

With that in place, we can finally deploy our ranking model.

In [ ]:
ranking_model = mr.get_best_model("ranking_model", "fscore", "max")
ranking_deployment = ranking_model.deploy(name="rankingdeployment", script_file=predictor_script_path, model_server="PYTHON", serving_tool="KSERVE")

Let's test that it works.

In [ ]:
ranking_deployment.start()

input_example = list(ranking_model.input_example[0].values()) # Model expects list, not dict.
ranking_deployment.predict({"inputs" : input_example}) # NOTE "inputs" instead of "instances"

In [ ]:
ranking_deployment.stop()